In [1]:
import os
os.chdir('..')

os.environ["DJANGO_SETTINGS_MODULE"] = "course_management.settings"
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
os.environ["IS_LOCAL"] = "1"

import django
django.setup()

IS_LOCAL=True
['C:\\Users\\alexe\\.virtualenvs\\course-management-platform-wiAsnpQu\\lib\\site-packages\\ipykernel_launcher.py', '-f', 'C:\\Users\\alexe\\AppData\\Roaming\\jupyter\\runtime\\kernel-4fa5bab0-9c8f-43e8-a32a-cc867114ed77.json']
Is test: False


In [2]:
from courses.models import *

In [3]:
from collections import defaultdict

import pandas as pd

from django.forms.models import model_to_dict
from tqdm.auto import tqdm

In [5]:
course = Course.objects.get(slug='de-zoomcamp-2024')
homework = Homework.objects.get(course=course, slug='leaderboard')

In [15]:
enrollments = Enrollment.objects.filter(course=course)
df_enrollments = pd.DataFrame(enrollments.values())

df_enrollments = df_enrollments.sort_values(by='position_on_leaderboard')
cutoff_score = df_enrollments.iloc[99].total_score
df_public_leaderboard = df_enrollments[df_enrollments.total_score >= cutoff_score]

In [27]:
data_by_enrollment = {}
enrollments = []

submissions = homework.submission_set.all()

for submission in tqdm(submissions):
    e = submission.enrollment

    submission_dict = {
        "answers": list(
            submission.answer_set.values(
                "question_id", "answer_text"
            )
        ),
    }

    data_by_enrollment[e.id] = submission_dict
    enrollments.append(e)

100%|████████████████████████████████████████████████████████████| 102/102 [00:09<00:00, 10.32it/s]


In [32]:
projects = ProjectSubmission.objects.filter(enrollment__in=enrollments)

projects_by_student = defaultdict(list)

for p in projects:
    projects_by_student[p.enrollment_id].append(p.github_link)

In [48]:
question_ids = {
    58: 'github',
    59: 'linkedin',
    60: 'include_projects',
    61: 'comment',
}

In [49]:
leaderboard = df_public_leaderboard.to_dict(orient='records')

In [87]:
records = []

for s in leaderboard:
    e_id = s['id']
    name = s['certificate_name'] or s['display_name']
    score = s['total_score']

    record = {
        'name': name,
        'score': score,
    }

    data = data_by_enrollment.get(e_id)

    if not data:
        record['name'] = s['display_name']
    if data:
        for a in data['answers']:
            q_id = a['question_id']
            if q_id not in question_ids:
                continue
        
            q = question_ids[q_id]
            
            if q == 'include_projects' and a['answer_text'] == '1':
                record['projects'] = projects_by_student[e_id]
            else:
                record[q] = a['answer_text'] 

    records.append(record)


In [88]:
records[:5]

[{'name': 'Ashraf Mohammad',
  'score': 219,
  'github': 'www.github.com/Ashraf1395',
  'linkedin': 'www.linkedin.com/in/ashraf1395',
  'projects': ['https://github.com/Ashraf1395/customer_retention_analytics',
   'https://github.com/Ashraf1395/supply_chain_finance.git'],
  'comment': 'Really Recommend this bootcamp , if you want to get hands on data engineering experience.     My two Capstone project: www.github.com/Ashraf1395/supply_chain_finance, www.github.com/Ashraf1395/customer_retention_analytics'},
 {'name': 'Jorge Vladimir Abrego Arevalo',
  'score': 212,
  'github': 'https://github.com/JorgeAbrego',
  'linkedin': 'https://www.linkedin.com/in/jorge-abrego/',
  'projects': ['https://github.com/JorgeAbrego/weather_stream_project',
   'https://github.com/JorgeAbrego/capital_bikeshare_project'],
  'comment': ''},
 {'name': 'Purnendu Shekhar Shukla', 'score': 194},
 {'name': 'Krishna Anand',
  'score': 188,
  'projects': ['https://github.com/anandaiml19/DE_Zoomcamp_Project2/tree/ma

In [89]:
icon_prefix = 'https://user-images.githubusercontent.com/875246'

github_icon_url = f'{icon_prefix}/192300611-a606521b-cb76-4090-be8e-7cc21752b996.png'
linkedin_icon_url = f'{icon_prefix}/192300614-2ce22ed5-bbc4-4684-8098-d8128d71aac5.png'

github_icon = f'<img src="{github_icon_url}" height="16em" />'
linkedin_icon = f'<img src="{linkedin_icon_url}" height="16em" />'

In [94]:
f_out = open('notebooks/leaderboard.md', 'w', encoding='utf-8')

f_out.write(f"""## Leaderboard 

This is the top [100 leaderboard](https://courses.datatalks.club/{course.slug}/leaderboard)
of participants of {course.title} edition!

<table>
<tr>
  <th>Name</th>
  <th>Projects</th>
  <th>Social</th>
  <th>Comments</th>
</tr>
""")

265

In [95]:
for record in records:
    line = '<tr>\n'

    line = line + f'  <td>{record["name"]}</td>\n'

    projects = record.get('projects', [])

    projects_line = ''

    for p in projects:
        projects_line += f' <a href="{p}">{github_icon}</a>'
    
    line = line + f'  <td>{projects_line}</a></td>\n'

    social = ''
    
    if record.get("linkedin"):
        social = social + f' <a href="{record["linkedin"]}">{linkedin_icon}</a>'

    if record.get("github"):
        social = social + f' <a href="{record["github"]}">{github_icon}</a>'

    line = line + f'  <td>{social}</td>\n'


    comment = ''

    if record.get("comment"):
        comment = f'<details>\n<summary>comment</summary>\n{record["comment"]}\n</details>'

    line = line + f'  <td>{comment}</td>\n'
    line = line + '</tr>\n'

    f_out.write(line)

f_out.write('</table>\n')

f_out.close()